In [1]:
from __future__ import annotations

from pathlib import Path
from datetime import datetime

OVERWRITE = True

def write_with_backup(path: str, content: str) -> None:
    p = Path(path)
    p.parent.mkdir(parents=True, exist_ok=True)
    if p.exists() and OVERWRITE:
        ts = datetime.now().strftime("%Y%m%d_%H%M%S")
        bak = p.with_suffix(p.suffix + f".bak_{ts}")
        bak.write_text(p.read_text(encoding="utf-8"), encoding="utf-8")
        print("BACKUP:", bak)
    p.write_text(content, encoding="utf-8")
    print("WROTE:", p)

# Ensure dirs
Path("src/careeragent/api").mkdir(parents=True, exist_ok=True)
Path("app/ui").mkdir(parents=True, exist_ok=True)
Path("src/careeragent/api/__init__.py").write_text("", encoding="utf-8")
Path("app/ui/__init__.py").write_text("", encoding="utf-8")


API_MAIN = r'''
from __future__ import annotations

# --- CareerOS-style bootstrap so uvicorn can import src-layout reliably ---
import sys
from pathlib import Path

ROOT_DIR = Path(__file__).resolve().parents[3]   # repo root
SRC_DIR = ROOT_DIR / "src"
if str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))

import json
from typing import Any, Dict

from fastapi import FastAPI, File, Form, HTTPException, UploadFile
from pydantic import BaseModel, Field

from careeragent.orchestration import ENGINE


app = FastAPI(title="CareerAgent-AI Beta Brain", version="0.3.0")


@app.get("/health")
def health() -> Dict[str, Any]:
    """
    Description: Health endpoint for UI connectivity checks.
    Layer: L0
    Input: None
    Output: ok + version
    """
    return {"status": "ok", "service": "careeragent-api", "version": "0.3.0"}


class AnalyzeResponse(BaseModel):
    """
    Description: /analyze response.
    Layer: L1
    Input: resume upload + preferences
    Output: run_id + status
    """
    run_id: str
    status: str


class ActionRequest(BaseModel):
    """
    Description: HITL action payload.
    Layer: L5
    Input: action_type + payload
    Output: state transition
    """
    action_type: str = Field(..., description="approve_ranking | reject_ranking | approve_drafts | reject_drafts")
    payload: Dict[str, Any] = Field(default_factory=dict)


@app.post("/analyze", response_model=AnalyzeResponse)
async def analyze(
    resume: UploadFile = File(...),
    preferences_json: str = Form(...),
) -> AnalyzeResponse:
    """
    Description: One-click analyze. Accepts PDF/TXT/DOCX resume + preferences. No manual resume text.
    Layer: L1
    Input: multipart form
    Output: run_id
    """
    try:
        prefs = json.loads(preferences_json)
    except Exception as e:
        raise HTTPException(status_code=400, detail=f"Invalid preferences_json: {e}")

    data = await resume.read()
    if not data:
        raise HTTPException(status_code=400, detail="Empty resume upload.")

    st = ENGINE.start_run(filename=resume.filename or "resume", data=data, prefs=prefs)
    return AnalyzeResponse(run_id=st.run_id, status=st.status)


@app.get("/status/{run_id}")
def status(run_id: str) -> Dict[str, Any]:
    """
    Description: Poll current OrchestrationState from local SQLite store.
    Layer: L1
    Input: run_id
    Output: state JSON dict
    """
    st = ENGINE.load(run_id)
    if not st:
        raise HTTPException(status_code=404, detail="run_id not found")
    return st


@app.post("/action/{run_id}")
def action(run_id: str, req: ActionRequest) -> Dict[str, Any]:
    """
    Description: Submit HITL decision and continue automation.
    Layer: L5
    Input: action_type + payload
    Output: updated state JSON
    """
    try:
        st = ENGINE.submit_action(run_id=run_id, action_type=req.action_type, payload=req.payload)
    except ValueError:
        raise HTTPException(status_code=404, detail="run_id not found")
    return st.model_dump()
'''
write_with_backup("src/careeragent/api/main.py", API_MAIN)


DASHBOARD = r'''
from __future__ import annotations

import json
import os
from pathlib import Path
from typing import Any, Dict, Optional

import requests
import streamlit as st

# Default backend URL (CareerOS style: ENV -> secrets -> fallback)
DEFAULT_API = os.getenv("API_URL", "http://127.0.0.1:8000")


def api_get(api_base: str, path: str, timeout: int = 10) -> requests.Response:
    return requests.get(f"{api_base}{path}", timeout=timeout)


def api_post(api_base: str, path: str, **kwargs) -> requests.Response:
    return requests.post(f"{api_base}{path}", timeout=kwargs.pop("timeout", 30), **kwargs)


def safe_json(resp: requests.Response) -> Dict[str, Any]:
    try:
        return resp.json()
    except Exception:
        return {"status": "error", "raw": resp.text[:800]}


def render_status_badge(api_base: str) -> None:
    with st.sidebar:
        st.divider()
        st.subheader("System Status")
        try:
            r = api_get(api_base, "/health", timeout=3)
            if r.status_code == 200:
                st.success("🟢 Backend Online")
                st.caption(api_base)
            else:
                st.warning(f"🟠 Backend Issue ({r.status_code})")
        except Exception as e:
            st.error("🔴 Backend Offline")
            st.caption(str(e))


def render_live_feed(feed: list[dict]) -> None:
    if not feed:
        st.info("No live feed yet.")
        return
    for ev in feed[-140:]:
        st.write(f"**[{ev.get('layer')} {ev.get('agent')}]** {ev.get('message')}")


def file_exists(p: Optional[str]) -> bool:
    return bool(p) and Path(p).exists()


def download_button(label: str, path: str, mime: str) -> None:
    p = Path(path)
    if not p.exists():
        st.warning(f"Missing file: {path}")
        return
    st.download_button(label, data=p.read_bytes(), file_name=p.name, mime=mime, use_container_width=True)


def main() -> None:
    st.set_page_config(page_title="CareerAgent-AI Mission Control", layout="wide")
    st.title("CareerAgent-AI — Mission Control (One-Click Automation)")
    st.caption("Upload resume → autonomous ingestion + discovery + ranking → HITL approvals → drafts + dossier downloads")

    api_base = st.sidebar.text_input("API Base URL", value=DEFAULT_API)
    render_status_badge(api_base)

    st.sidebar.subheader("Resume Upload (no manual text)")
    resume_file = st.sidebar.file_uploader("Upload Resume", type=["pdf", "txt", "docx"])

    st.sidebar.subheader("Preferences")
    target_role = st.sidebar.text_input("Target role", value="Data Scientist")
    country = st.sidebar.text_input("Country", value="US")
    location = st.sidebar.text_input("Location", value="United States")
    remote = st.sidebar.checkbox("Remote preferred", value=True)
    wfo = st.sidebar.checkbox("On-site/WFO acceptable", value=True)
    salary = st.sidebar.text_input("Salary target (optional)", value="")
    user_phone = st.sidebar.text_input("Phone for SMS (optional)", value="")

    st.sidebar.subheader("Autonomy Controls")
    discovery_threshold = st.sidebar.slider("Discovery confidence threshold", 0.50, 0.90, 0.70, 0.05)
    max_refinements = st.sidebar.slider("Max query refinements", 1, 5, 3, 1)

    run_btn = st.sidebar.button("🚀 RUN ONE-CLICK", type="primary", use_container_width=True, disabled=(resume_file is None))

    st.sidebar.subheader("Existing Run")
    run_id_in = st.sidebar.text_input("Run ID", value=st.session_state.get("run_id", ""))

    if run_btn:
        prefs = {
            "target_role": target_role,
            "country": country,
            "location": location,
            "remote": remote,
            "wfo_ok": wfo,
            "salary": salary,
            "user_phone": user_phone.strip() or None,
            "discovery_threshold": float(discovery_threshold),
            "max_refinements": int(max_refinements),
        }

        files = {"resume": (resume_file.name, resume_file.getvalue())}
        data = {"preferences_json": json.dumps(prefs)}

        try:
            r = api_post(api_base, "/analyze", files=files, data=data, timeout=120)
        except Exception as e:
            st.error(f"❌ API not reachable: {e}")
            return

        if r.status_code >= 400:
            st.error(f"/analyze failed: {r.status_code}\n\n{r.text[:1200]}")
            return

        out = safe_json(r)
        st.session_state["run_id"] = out["run_id"]
        st.success(f"Run started: {out['run_id']} (status: {out['status']})")

    run_id = st.session_state.get("run_id") or run_id_in.strip()
    if not run_id:
        st.info("Upload a resume and click RUN, or paste an existing run_id.")
        return

    # Poll state
    colA, colB, colC = st.columns([1, 1, 1])
    refresh = st.button("🔄 Refresh", use_container_width=True)

    try:
        r = api_get(api_base, f"/status/{run_id}", timeout=20)
        if r.status_code != 200:
            st.warning(f"Run not found yet ({r.status_code}).")
            return
        state = safe_json(r)
    except Exception as e:
        st.error(f"❌ API not reachable: {e}")
        return

    status = state.get("status", "unknown")
    meta = state.get("meta", {}) or {}
    steps = state.get("steps", []) or []
    feed = meta.get("live_feed", []) or []
    artifacts = state.get("artifacts", {}) or {}
    pending = meta.get("pending_action")

    with colA:
        st.metric("Run ID", run_id)
    with colB:
        st.metric("Status", status)
    with colC:
        st.metric("Pending", str(pending))

    # Progress estimate
    total = max(1, len(steps))
    done = sum(1 for s in steps if s.get("finished_at_utc"))
    st.progress(done / total)

    left, right = st.columns([1.2, 0.8], vertical_alignment="top")

    with left:
        st.markdown("### Live Agent Feed")
        render_live_feed(feed)

    with right:
        st.markdown("### Key Scores")
        job_scores = meta.get("job_scores", {}) or {}
        if job_scores:
            jid, score = next(iter(job_scores.items()))
            pct = float(score) * 100.0
            st.metric("Top InterviewChance", f"{pct:.2f}%")
            st.progress(min(1.0, max(0.0, pct / 100.0)))
        else:
            st.caption("Scores not available yet.")

        st.markdown("### Downloads")
        zip_ref = artifacts.get("career_dossier_zip")
        pdf_ref = artifacts.get("xai_transparency_pdf")

        if zip_ref and file_exists(zip_ref.get("path")):
            download_button("Download Career Dossier (ZIP)", zip_ref["path"], "application/zip")
        else:
            st.caption("Dossier ZIP not ready.")

        if pdf_ref and file_exists(pdf_ref.get("path")):
            download_button("Download XAI Transparency (PDF)", pdf_ref["path"], "application/pdf")
        else:
            st.caption("XAI PDF not ready.")

    st.markdown("### Ranking")
    ranking_ref = artifacts.get("ranking")
    ranking = None
    if ranking_ref and file_exists(ranking_ref.get("path")):
        ranking = json.loads(Path(ranking_ref["path"]).read_text(encoding="utf-8"))
        st.dataframe(
            [{
                "score_%": round(float(x.get("overall_match_percent", 0.0)), 2),
                "role": x.get("role_title"),
                "company": x.get("company"),
                "source": x.get("source"),
                "url": x.get("url"),
                "missing_required": ", ".join((x.get("missing_required_skills") or [])[:5]),
            } for x in ranking],
            use_container_width=True
        )
    else:
        st.caption("Ranking not available yet.")

    st.markdown("### Human-in-the-Loop Controls")
    if status == "needs_human_approval" and pending == "review_ranking":
        c1, c2 = st.columns(2)
        with c1:
            if st.button("✅ Approve Ranking → Generate Drafts", type="primary", use_container_width=True):
                rr = api_post(api_base, f"/action/{run_id}", json={"action_type": "approve_ranking", "payload": {}}, timeout=30)
                st.success("Approved. Draft generation started.")
        with c2:
            reason = st.text_input("Reason to refine ranking", value="")
            if st.button("❌ Reject Ranking → Re-run Discovery", use_container_width=True):
                rr = api_post(api_base, f"/action/{run_id}", json={"action_type": "reject_ranking", "payload": {"reason": reason}}, timeout=30)
                st.warning("Rejected. Discovery refinement started.")

    if status == "needs_human_approval" and pending == "review_drafts":
        c1, c2 = st.columns(2)
        with c1:
            if st.button("✅ Approve Drafts → Submit (Simulated)", type="primary", use_container_width=True):
                rr = api_post(api_base, f"/action/{run_id}", json={"action_type": "approve_drafts", "payload": {}}, timeout=30)
                st.success("Approved. Submissions started.")
        with c2:
            reason = st.text_input("Reason to revise drafts", value="")
            if st.button("❌ Reject Drafts → Back to Ranking", use_container_width=True):
                rr = api_post(api_base, f"/action/{run_id}", json={"action_type": "reject_drafts", "payload": {"reason": reason}}, timeout=30)
                st.warning("Rejected drafts. Returning to ranking review.")

    st.markdown("### Drafts Bundle (preview)")
    drafts_ref = artifacts.get("drafts_bundle")
    if drafts_ref and file_exists(drafts_ref.get("path")):
        bundle = json.loads(Path(drafts_ref["path"]).read_text(encoding="utf-8"))
        st.json(bundle)
    else:
        st.caption("Draft bundle not available yet.")

    with st.expander("Full State JSON"):
        st.json(state)


if __name__ == "__main__":
    main()
'''
write_with_backup("app/ui/dashboard.py", DASHBOARD)


APP_MAIN = r'''
from __future__ import annotations

import sys
from pathlib import Path

# Ensure src/ is visible for local streamlit runs
ROOT = Path(__file__).resolve().parents[1]
SRC = ROOT / "src"
if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))

from app.ui.dashboard import main

main()
'''
write_with_backup("app/main.py", APP_MAIN)

# Compatibility wrapper if you still run: streamlit run app/dashboard.py
APP_DASH = r'''
from __future__ import annotations

import sys
from pathlib import Path

ROOT = Path(__file__).resolve().parents[1]
SRC = ROOT / "src"
if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))

from app.ui.dashboard import main

main()
'''
write_with_backup("app/dashboard.py", APP_DASH)

print("\n✅ UI now supports PDF/TXT/DOCX upload (no resume text box).")
print("Next run commands:")
print("1) API:      PYTHONPATH=src uv run python -m uvicorn careeragent.api.main:app --host 127.0.0.1 --port 8000 --reload")
print("2) Streamlit:PYTHONPATH=src uv run streamlit run app/main.py --server.port 8501")


WROTE: src/careeragent/api/main.py
BACKUP: app/ui/dashboard.py.bak_20260220_225827
WROTE: app/ui/dashboard.py
BACKUP: app/main.py.bak_20260220_225827
WROTE: app/main.py
WROTE: app/dashboard.py

✅ UI now supports PDF/TXT/DOCX upload (no resume text box).
Next run commands:
1) API:      PYTHONPATH=src uv run python -m uvicorn careeragent.api.main:app --host 127.0.0.1 --port 8000 --reload
2) Streamlit:PYTHONPATH=src uv run streamlit run app/main.py --server.port 8501
